In [12]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
from tqdm.notebook import tqdm
from baseline_utils import *

In [49]:
# create a dataframe to store our metrics
logs = pd.DataFrame(data=None, columns=["embed_size", "average_train_time", "trainable_params"])

# load all of our baseline models -- get parameter counts + average train time
for embed_size in [192, 384, 576, 768, 960]:
    
    # load in our time-elapsed logs (for TRAINING ONLY) - threshold to 99% percentile to ignore validation sets
    foldername = f"char_models/embed-size={embed_size}"
    times = pd.read_csv(f"{foldername}/logs.csv").wallclock.values
    average_train_time = times[times <= np.quantile(times, q=0.99)].mean()
    
    # load in the model and get our weights
    model = torch.load(f"{foldername}/model.pth")
    trainable_params = np.sum(
        [param.numel() for param in model.parameters()
         if param.requires_grad])
    
    # record in our table
    logs.loc[len(logs.index)] = [embed_size, average_train_time, trainable_params]
    
# save as a .csv
logs.to_csv("baseline_logs.csv", index=False)